In [1]:
import os
import pandas as pd
import numpy as np
import torch
from torch import nn
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

# READING AND LOADING DATA

In [2]:
file_path = "../Data/Raw/"

In [3]:
data_csv_list = os.listdir(file_path)
data_csv_list

['pharmacy.csv',
 '.DS_Store',
 'encounters.csv',
 'loinc.csv',
 'demographics.csv']

In [4]:
pharmacy_raw_df = pd.read_csv(file_path+"/"+"pharmacy.csv")
pharmacy_raw_df

,STUDY_ID,REFILL_TIME,NDC,GPI_GENERIC_NAME,NAME_DRG_GROUP,NAME_DRG_CLASS,NAME_DRG_SUB_CLASS,NUMBER_OF_DAYS_SUPPLY,DISPENSE_AMOUNT,GIVE_STRENGTH
0,19492,18MAY2017,46087221,"Estrogens, Conjugated Vaginal Cream 0.625 MG/GM",VAGINAL AND RELATED PRODUCTS,Vaginal Estrogens,Vaginal Estrogens,30,30.0,0.625
1,19492,31JUL2017,13107015490,Paroxetine HCl Tab 10 MG,ANTIDEPRESSANTS,Selective Serotonin Reuptake Inhibitors (SSRIs),Selective Serotonin Reuptake Inhibitors (SSRIs),30,30.0,10.000
2,19492,14AUG2017,43547036009,Losartan Potassium Tab 25 MG,ANTIHYPERTENSIVES,Angiotensin II Receptor Antagonists,Angiotensin II Receptor Antagonists,30,30.0,25.000
3,19492,28AUG2017,13107015490,Paroxetine HCl Tab 10 MG,ANTIDEPRESSANTS,Selective Serotonin Reuptake Inhibitors (SSRIs),Selective Serotonin Reuptake Inhibitors (SSRIs),30,30.0,10.000
4,19492,11SEP2017,43547036009,Losartan Potassium Tab 25 MG,ANTIHYPERTENSIVES,Angiotensin II Receptor Antagonists,Angiotensin II Receptor Antagonists,30,30.0,25.000
...,...,...,...,...,...,...,...,...,...,...
58325,301575470,19NOV2018,55111029336,Sumatriptan Succinate Tab 100 MG,MIGRAINE PRODUCTS,Serotonin Agonists,Selective Serotonin Agonists 5-HT(1),30,9.0,100.000
58326,301575470,14DEC2018,55111029336,Sumatriptan Succinate Tab 100 MG,MIGRAINE PRODUCTS,Serotonin Agonists,Selective Serotonin Agonists 5-HT(1),30,9.0,100.000
58327,301575470,28JAN2019,55111029336,Sumatriptan Succinate Tab 100 MG,MIGRAINE PRODUCTS,Serotonin Agonists,Selective Serotonin Agonists 5-HT(1),5,9.0,100.000
58328,301575470,29JAN2019,378181377,Levothyroxine Sodium Tab 125 MCG,THYROID AGENTS,Thyroid Hormones,Thyroid Hormones,90,90.0,125.000


In [5]:
encounters_raw_df = pd.read_csv(file_path+"/"+"encounters.csv")
encounters_raw_df

,STUDY_ID,ADMIT_TIME,DISCHARGE_TIME,ENC_TYPE,DX_SYS,DX_CODE,DESCRIPTION
0,19492,16MAY2016,16MAY2016,O,ICD10,Z12.31,ENCNTR SCREEN MAMMOGRAM FOR MALIGNANT NEOPLASM...
1,19492,16MAY2016,16MAY2016,O,ICD10,Z80.3,FAMILY HISTORY OF MALIGNANT NEOPLASM OF BREAST
2,19492,31OCT2016,31OCT2016,O,ICD10,F40.10,"SOCIAL PHOBIA, UNSPECIFIED"
3,19492,31OCT2016,31OCT2016,O,ICD10,F40.11,"SOCIAL PHOBIA, GENERALIZED"
4,19492,31OCT2016,31OCT2016,O,ICD10,I34.0,NONRHEUMATIC MITRAL VALVE INSUFFICIENCY
...,...,...,...,...,...,...,...
95966,308250127,02NOV2019,02NOV2019,E,ICD10,M72.2,PLANTAR FASCIAL FIBROMATOSIS
95967,308250127,02NOV2019,02NOV2019,E,ICD10,M77.30,"CALCANEAL SPUR, UNSPECIFIED FOOT"
95968,308250127,02NOV2019,02NOV2019,E,ICD10,M79.89,OTHER SPECIFIED SOFT TISSUE DISORDERS
95969,308250127,16JAN2020,16JAN2020,O,NaN,UNKNOWN/NOT AVAILABLE,NaN


In [6]:
demographic_raw_df = pd.read_csv(file_path+"/"+"demographics.csv")
demographic_raw_df

,STUDY_ID,DATEOFSTUDY,DOB,AGE,GENDER,RACE,ETHNICITY,HEIGHT,WEIGHT,BMI,PROSTHESIS,PROSTHESISPLEASESPECIFY,ORTHOSIS,ORTHOSIS_PLEASESPECIFY_,COMMENTS,BESTGRIPSTRENGTH,TIMEFOR5CHAIRSTANDS,NORLANDAPPENDICULARLEAN,USUALGAITSPEED,TOTALSPPBSCORE
0,19492,07NOV2019,07FEB1957,62.75,F,White,Not Hispanic or Latino,170.1,69.6,24.1,0,No,0,0,0,0,0,0,0,0
1,20956,21OCT2019,30JUN1951,68.31,F,Black or African American,Not Hispanic or Latino,164.9,138.4,50.9,0,No,0,0,0,0,1,0,0,0
2,25874,05OCT2018,11DEC1958,59.82,F,Black or African American,Non-Hispanic,179.6,120.7,37.4,No,0,No,0,0,0,1,0,0,1
3,40532,19APR2019,14SEP1943,75.59,F,White,Non-Hispanic,151.4,57.0,24.9,No,0,No,0,0,1,0,0,0,0
4,41155,11JUL2018,23FEB1953,65.38,F,White,Non-Hispanic,158.9,58.0,23.0,No,0,No,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1299,285854405,28NOV2018,22NOV1987,31.02,F,White,Non-Hispanic,163.0,112.8,42.5,No,0,No,0,0,0,0,0,0,0
1300,286002457,18JUL2018,17AUG1968,49.92,F,Asian,Non-Hispanic,150.8,51.8,22.8,No,0,No,0,0,0,0,1,0,0
1301,288993296,22FEB2019,05MAY1955,63.80,F,Asian,Non-Hispanic,166.8,65.4,23.5,Yes,LEFT HIP REPLACEMENT,No,0,0,0,1,0,0,1
1302,301575470,17DEC2019,22FEB1962,57.82,F,White,Not Hispanic or Latino,158.6,66.8,26.6,"Plate, 4 screws in Rt. arm; clips on breast",Yes,shoe inserts,0,0,0,0,0,0,0


# Cleaning Data

In [7]:
#demographic_X_train.isna().sum() #check na's in each columns

In [8]:
demographic_X_df = demographic_raw_df.iloc[:, 0:15]
target_Y_df = demographic_raw_df.iloc[:, 15:] #last 5 columns are various tests that indicate sarcopenia. Translate later
demographic_X_df

,STUDY_ID,DATEOFSTUDY,DOB,AGE,GENDER,RACE,ETHNICITY,HEIGHT,WEIGHT,BMI,PROSTHESIS,PROSTHESISPLEASESPECIFY,ORTHOSIS,ORTHOSIS_PLEASESPECIFY_,COMMENTS
0,19492,07NOV2019,07FEB1957,62.75,F,White,Not Hispanic or Latino,170.1,69.6,24.1,0,No,0,0,0
1,20956,21OCT2019,30JUN1951,68.31,F,Black or African American,Not Hispanic or Latino,164.9,138.4,50.9,0,No,0,0,0
2,25874,05OCT2018,11DEC1958,59.82,F,Black or African American,Non-Hispanic,179.6,120.7,37.4,No,0,No,0,0
3,40532,19APR2019,14SEP1943,75.59,F,White,Non-Hispanic,151.4,57.0,24.9,No,0,No,0,0
4,41155,11JUL2018,23FEB1953,65.38,F,White,Non-Hispanic,158.9,58.0,23.0,No,0,No,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1299,285854405,28NOV2018,22NOV1987,31.02,F,White,Non-Hispanic,163.0,112.8,42.5,No,0,No,0,0
1300,286002457,18JUL2018,17AUG1968,49.92,F,Asian,Non-Hispanic,150.8,51.8,22.8,No,0,No,0,0
1301,288993296,22FEB2019,05MAY1955,63.80,F,Asian,Non-Hispanic,166.8,65.4,23.5,Yes,LEFT HIP REPLACEMENT,No,0,0
1302,301575470,17DEC2019,22FEB1962,57.82,F,White,Not Hispanic or Latino,158.6,66.8,26.6,"Plate, 4 screws in Rt. arm; clips on breast",Yes,shoe inserts,0,0


In [9]:
#We have 5 tests that indicate sarcopenia. If 1 in any 5 test = 1 in Sarcopenia Colum
target_Y_df["SARCOPENIA"] = target_Y_df.sum(axis = 1)
target_Y_df

,BESTGRIPSTRENGTH,TIMEFOR5CHAIRSTANDS,NORLANDAPPENDICULARLEAN,USUALGAITSPEED,TOTALSPPBSCORE,SARCOPENIA
0,0,0,0,0,0,0
1,0,1,0,0,0,1
2,0,1,0,0,1,2
3,1,0,0,0,0,1
4,1,0,1,0,0,2
...,...,...,...,...,...,...
1299,0,0,0,0,0,0
1300,0,0,1,0,0,1
1301,0,1,0,0,1,2
1302,0,0,0,0,0,0


## Mapping raw age to different age groups

In [10]:
def encoding_age_group(age):
    #This function map age to age groups. Then maps age groups to integer encoding
    
    #Example: 
    
    #5 -> < 18 -> 0
    #23 -> [18: 29] -> 1
    #34 -> [30:39] -> 2
    
    #
    age_range = [(0,18), (18, 29), (29,39), (39, 49), (49,59), (59,69), (69,79), (79,89), (89,99), (99, float("inf"))]
    for i, range_ in enumerate(age_range):
        if age >= range_[0] and age < range_[1]:
            return i

In [11]:
demographic_X_df["AGE_GROUP"] = demographic_X_df["AGE"].apply(lambda x: encoding_age_group(x))

## Encoding Gender (F, M) -> (0,1), respectively

In [12]:
demographic_X_df["GENDER"] = demographic_X_df["GENDER"].apply(lambda x: 0 if x == "F" else 1)

## Encoding Race and Ethnicity 

In [13]:
race_arr = demographic_X_df["RACE"].unique()
def encoding_race(v):
    for i, race in enumerate(race_arr):
        if v == race:
            return i

In [14]:
demographic_X_df["RACE"] = demographic_X_df["RACE"].apply(lambda x: encoding_race(x))

In [15]:
eth_arr = demographic_X_df["ETHNICITY"].unique()
def encoding_ethnicity(v):
    for i, eth in enumerate(eth_arr):
        if v == eth:
            return i

In [16]:
demographic_X_df["ETHNICITY"] = demographic_X_df["ETHNICITY"].apply(lambda x: encoding_ethnicity(x))

## Encoding classification label

In [17]:
demographic_X_df["SARCOPENIA"] =  target_Y_df["SARCOPENIA"] > 0
demographic_X_df["SARCOPENIA"] = demographic_X_df["SARCOPENIA"].astype(int)

In [18]:
demographic_X_df

,STUDY_ID,DATEOFSTUDY,DOB,AGE,GENDER,RACE,ETHNICITY,HEIGHT,WEIGHT,BMI,PROSTHESIS,PROSTHESISPLEASESPECIFY,ORTHOSIS,ORTHOSIS_PLEASESPECIFY_,COMMENTS,AGE_GROUP,SARCOPENIA
0,19492,07NOV2019,07FEB1957,62.75,0,0,0,170.1,69.6,24.1,0,No,0,0,0,5,0
1,20956,21OCT2019,30JUN1951,68.31,0,1,0,164.9,138.4,50.9,0,No,0,0,0,5,1
2,25874,05OCT2018,11DEC1958,59.82,0,1,1,179.6,120.7,37.4,No,0,No,0,0,5,1
3,40532,19APR2019,14SEP1943,75.59,0,0,1,151.4,57.0,24.9,No,0,No,0,0,6,1
4,41155,11JUL2018,23FEB1953,65.38,0,0,1,158.9,58.0,23.0,No,0,No,0,0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1299,285854405,28NOV2018,22NOV1987,31.02,0,0,1,163.0,112.8,42.5,No,0,No,0,0,2,0
1300,286002457,18JUL2018,17AUG1968,49.92,0,2,1,150.8,51.8,22.8,No,0,No,0,0,4,1
1301,288993296,22FEB2019,05MAY1955,63.80,0,2,1,166.8,65.4,23.5,Yes,LEFT HIP REPLACEMENT,No,0,0,5,1
1302,301575470,17DEC2019,22FEB1962,57.82,0,0,0,158.6,66.8,26.6,"Plate, 4 screws in Rt. arm; clips on breast",Yes,shoe inserts,0,0,4,0


In [19]:
demographic_X_trimmed = demographic_X_df[["STUDY_ID", "SARCOPENIA", "GENDER", "RACE", "BMI" , "AGE_GROUP"]]
demographic_X_trimmed

,STUDY_ID,SARCOPENIA,GENDER,RACE,BMI,AGE_GROUP
0,19492,0,0,0,24.1,5
1,20956,1,0,1,50.9,5
2,25874,1,0,1,37.4,5
3,40532,1,0,0,24.9,6
4,41155,1,0,0,23.0,5
...,...,...,...,...,...,...
1299,285854405,0,0,0,42.5,2
1300,286002457,1,0,2,22.8,4
1301,288993296,1,0,2,23.5,5
1302,301575470,0,0,0,26.6,4


## Train test split (80-20)

In [20]:
from sklearn.model_selection import train_test_split

demographic_train, demographic_test = train_test_split(demographic_X_trimmed, test_size=0.2, random_state = 12)
Y_train = demographic_train["SARCOPENIA"]
Y_test = demographic_test["SARCOPENIA"]
demographic_test

,STUDY_ID,SARCOPENIA,GENDER,RACE,BMI,AGE_GROUP
477,3652678,1,0,1,60.0,6
1061,42136830,0,0,0,24.4,2
27,143215,1,0,0,23.9,7
814,7195996,0,0,0,20.0,2
374,3293091,0,1,0,22.0,6
...,...,...,...,...,...,...
363,3260152,0,0,0,32.1,5
151,728094,0,0,1,23.8,6
735,5871662,0,0,0,34.4,3
387,3359539,0,0,2,25.8,5


## Min Maxing demographics data

In [21]:
for col in demographic_train.columns[2:]:
    #do test first cause train will be zeroed out
    demographic_test[col]=(demographic_test[col]-demographic_train[col].min())/(demographic_train[col].max()-demographic_train[col].min())
    demographic_train[col]=(demographic_train[col]-demographic_train[col].min())/(demographic_train[col].max()-demographic_train[col].min())
demographic_train = demographic_train[["STUDY_ID", "GENDER", "RACE", "BMI", "AGE_GROUP"]]
demographic_test = demographic_test[["STUDY_ID", "GENDER", "RACE", "BMI", "AGE_GROUP"]]
demographic_test

,STUDY_ID,GENDER,RACE,BMI,AGE_GROUP
477,3652678,0.0,0.142857,0.855491,0.714286
1061,42136830,0.0,0.000000,0.169557,0.142857
27,143215,0.0,0.000000,0.159923,0.857143
814,7195996,0.0,0.000000,0.084778,0.142857
374,3293091,1.0,0.000000,0.123314,0.714286
...,...,...,...,...,...
363,3260152,0.0,0.000000,0.317919,0.571429
151,728094,0.0,0.142857,0.157996,0.714286
735,5871662,0.0,0.000000,0.362235,0.285714
387,3359539,0.0,0.285714,0.196532,0.571429


In [22]:
id_demographics_train = dict()
id_demographics_test = dict()
for index, row in demographic_train.iterrows():
    patient_id = int(row[0])
    id_demographics_train[patient_id] = np.zeros(demographic_train.shape[1]-1) #substact the id column
    for i in range(1, demographic_train.shape[1]):
        id_demographics_train[patient_id][i-1] = row[i]    

for index, row in demographic_test.iterrows():
    patient_id = int(row[0])
    id_demographics_test[patient_id] = np.zeros(demographic_test.shape[1]-1) #substact the id column
    for i in range(1, demographic_test.shape[1]):
        id_demographics_test[patient_id][i-1] = row[i]    

In [23]:
encounters_raw_df.isna().sum()

STUDY_ID              0
ADMIT_TIME            0
DISCHARGE_TIME        0
ENC_TYPE              0
DX_SYS            10670
DX_CODE               0
DESCRIPTION       10789
dtype: int64

## Simplify and Translate DX_Code to Integer encoding

#### Simply DX_CODE

In [24]:
def simplify_DX_CODE(v):
    #return major category
    arr = v.split(".")
    return arr[0]

In [25]:
encounters_raw_df = encounters_raw_df.dropna() #drop any row with Na's. Mostly DX_SYS and DX_CODE

In [26]:
encounters_raw_df["DX_CODE"] = encounters_raw_df["DX_CODE"].apply(lambda x: simplify_DX_CODE(x))

/var/folders/72/zllm1_k55v9809qz_mgkcq6m0000gn/T/ipykernel_12110/157681508.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  encounters_raw_df["DX_CODE"] = encounters_raw_df["DX_CODE"].apply(lambda x: simplify_DX_CODE(x))


In [27]:
# encounters_raw_df.DX_CODE.unique().size
# After simplifying, we still have more than 1000 unique codes.
# Further reduction is needed

In [28]:
dx_group = pd.read_csv("ICD_10_CODE_2022.csv", header = None)
dx_group.columns = ["Disease_Family", 'Code_Group']
dx_group

,Disease_Family,Code_Group
0,A00-A09,Intestinal infectious diseases
1,A15-A19,Tuberculosis
2,A20-A28,Certain zoonotic bacterial diseases
3,A30-A49,Other bacterial diseases
4,A50-A64,Infections with a predominantly sexual mode of...
...,...,...
280,Z66-Z66,Do not resuscitate status
281,Z67-Z67,Blood type
282,Z68-Z68,Body mass index (BMI)
283,Z69-Z76,Persons encountering health services in other ...


In [29]:
code_group_dict = defaultdict(list)
for index, row in dx_group.iterrows():
    code_group = row[0]
    try:
        left_interval, right_interval = code_group.split('-')
        letter_code = left_interval[0]
        left = int(left_interval[1:])
        right = int(right_interval[1:])
        assert(letter_code ==right_interval[0]) #assert that the letter code is the same
        code_group_dict[letter_code].append((left, right, index)) #tuple is the range and the integer coding as well
    except:
        print(row)


Disease_Family                            C7A-C7A
Code_Group        Malignant neuroendocrine tumors
Name: 35, dtype: object
Disease_Family                            C7B-C7B
Code_Group        Secondary neuroendocrine tumors
Name: 36, dtype: object
Disease_Family                         D3A-D3A
Code_Group        Benign neuroendocrine tumors
Name: 41, dtype: object
Disease_Family                         I30-I5A
Code_Group        Other forms of heart disease
Name: 104, dtype: object
Disease_Family                                              O94-O9A
Code_Group        Other obstetric conditions, not elsewhere clas...
Name: 178, dtype: object
Disease_Family    X92-Y09
Code_Group        Assault
Name: 263, dtype: object


In [30]:
code_group_dict["C7A"] = 35
code_group_dict["C7B"] = 36
code_group_dict["D3A"] = 41
code_group_dict["I"].append((30,53, 104)) #https://www.icd10data.com/ICD10CM/Codes/I00-I99/I30-I5A
code_group_dict["O"].append((94,99,178))
code_group_dict["X"].append((92,99,263))
code_group_dict["Y"].append((0,9,263))
code_group_dict

defaultdict(list,
            {'A': [(0, 9, 0),
              (15, 19, 1),
              (20, 28, 2),
              (30, 49, 3),
              (50, 64, 4),
              (65, 69, 5),
              (70, 74, 6),
              (75, 79, 7),
              (80, 89, 8),
              (90, 99, 9)],
             'B': [(0, 9, 10),
              (10, 10, 11),
              (15, 19, 12),
              (20, 20, 13),
              (25, 34, 14),
              (35, 49, 15),
              (50, 64, 16),
              (65, 83, 17),
              (85, 89, 18),
              (90, 94, 19),
              (95, 97, 20),
              (99, 99, 21)],
             'C': [(0, 14, 22),
              (15, 26, 23),
              (30, 39, 24),
              (40, 41, 25),
              (43, 44, 26),
              (45, 49, 27),
              (50, 50, 28),
              (51, 58, 29),
              (60, 63, 30),
              (64, 68, 31),
              (69, 72, 32),
              (73, 75, 33),
              (76, 80, 34),


In [31]:
#add code group that failed to be processed
code_group_dict["C"].append((0,99,15))
code_group_dict["D"].append((0,49,15))

In [32]:
unprocessed_DX_CODE = set()
def encoding_DX_CODE(v):
    try:
        lettercode = v[0]
        number = int(v[1:])
        potential_ranges = code_group_dict[lettercode]
        for range_ in potential_ranges:
            min_ = range_[0]
            max_ = range_[1]
            if min_ <= number <= max_:
                return (int(range_[2]))
    except:
        unprocessed_DX_CODE.add(v)


In [33]:

# unique_DX_code_arr = encounters_raw_df["DX_CODE"].unique() 
# unique_DX_code_dict = dict() #make a DX_CODE to integer dictionary
# for i,v in enumerate(unique_DX_code_arr):
#     unique_DX_code_dict[v] = i
    
# def encoding_DX_CODE(v):
#     return unique_DX_code_dict[v]

encounters_raw_df["DX_CODE_ENCODED"] = encounters_raw_df["DX_CODE"].apply(lambda x: encoding_DX_CODE(x))
encounters_raw_df

/var/folders/72/zllm1_k55v9809qz_mgkcq6m0000gn/T/ipykernel_12110/352905809.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  encounters_raw_df["DX_CODE_ENCODED"] = encounters_raw_df["DX_CODE"].apply(lambda x: encoding_DX_CODE(x))


,STUDY_ID,ADMIT_TIME,DISCHARGE_TIME,ENC_TYPE,DX_SYS,DX_CODE,DESCRIPTION,DX_CODE_ENCODED
0,19492,16MAY2016,16MAY2016,O,ICD10,Z12,ENCNTR SCREEN MAMMOGRAM FOR MALIGNANT NEOPLASM...,270.0
1,19492,16MAY2016,16MAY2016,O,ICD10,Z80,FAMILY HISTORY OF MALIGNANT NEOPLASM OF BREAST,284.0
2,19492,31OCT2016,31OCT2016,O,ICD10,F40,"SOCIAL PHOBIA, UNSPECIFIED",64.0
3,19492,31OCT2016,31OCT2016,O,ICD10,F40,"SOCIAL PHOBIA, GENERALIZED",64.0
4,19492,31OCT2016,31OCT2016,O,ICD10,I34,NONRHEUMATIC MITRAL VALVE INSUFFICIENCY,104.0
...,...,...,...,...,...,...,...,...
95959,301575470,13JUL2020,13JUL2020,O,ICD10,Z15,GENETIC SUSCEPTIBILITY TO MALIGNANT NEOPLASM O...,271.0
95960,301575470,13JUL2020,13JUL2020,O,ICD10,Z90,ACQUIRED ABSENCE OF BILATERAL BREASTS AND NIPPLES,284.0
95966,308250127,02NOV2019,02NOV2019,E,ICD10,M72,PLANTAR FASCIAL FIBROMATOSIS,151.0
95967,308250127,02NOV2019,02NOV2019,E,ICD10,M77,"CALCANEAL SPUR, UNSPECIFIED FOOT",151.0


In [34]:
print("Unprocessed code in df: ",unprocessed_DX_CODE)
encounters_raw_df.loc[encounters_raw_df["DX_CODE"] == "C7A", "DX_CODE_ENCODED"] = 35 #from dict
encounters_raw_df.loc[encounters_raw_df["DX_CODE"] == "Z3A", "DX_CODE_ENCODED"] = 277 #Z3A is in same family as Z30-Z39
encounters_raw_df.loc[encounters_raw_df["DX_CODE"] == "M1A", "DX_CODE_ENCODED"] = 141 #M1A is in same family as M05-M14
encounters_raw_df.loc[encounters_raw_df["DX_CODE"] == "C7B", "DX_CODE_ENCODED"] = 36 #from dict
encounters_raw_df.loc[encounters_raw_df["DX_CODE"] == "D3A", "DX_CODE_ENCODED"] = 41 #from dict


Unprocessed code in df:  {'C7A', 'C7B', 'Z3A', 'M1A', 'D3A'}


In [35]:
encounters_encoded = encounters_raw_df[["STUDY_ID", "DX_CODE_ENCODED"]]
encounters_encoded = encounters_encoded.astype(int)

In [36]:
encounters_encoded

,STUDY_ID,DX_CODE_ENCODED
0,19492,270
1,19492,284
2,19492,64
3,19492,64
4,19492,104
...,...,...
95959,301575470,271
95960,301575470,284
95966,308250127,151
95967,308250127,151


In [37]:
from collections import Counter

In [38]:
def integer_encode_top_k_variable(df,col, k):
    """
    This function integer-encode the top k values in df[col]--(0, k-1). Everything else is encoded as k.
    
    Return df with newly integer encoded
    """
    top_k_value = Counter(df[col]).most_common(k) #most common k value, return sorted list of tuple (value, count)
    
    dic = {}
    for idx, tup in enumerate(top_k_value):  #put top k element in dict for fast look up
        value = tup[0]
        dic[value] = idx
    for index, row in df.iterrows():
        row_val = row[col]
        if row_val in dic: #check if value is in top 75
            df.loc[index, col] = dic[row_val]
        else: #default other encoding of value not in top 75
            df.loc[index, col] = k
    
    
    #some sumary statistic
    s = 0
    for tup in top_k_value:
        s+= tup[1]
    
    total_count = Counter(df[col]).total()
    coverage_ratio = s/total_count
    print("Top ", k, " labels cover ", coverage_ratio, "% of all the values in ", col)
    return df
    

In [39]:
encounters_encoded_top50 = integer_encode_top_k_variable(encounters_encoded, "DX_CODE_ENCODED", 50)

Top  75  labels cover  0.886924467610528 % of all the values in  DX_CODE_ENCODED


In [40]:
encounters_encoded_top50

,STUDY_ID,DX_CODE_ENCODED
0,19492,0
1,19492,1
2,19492,13
3,19492,13
4,19492,16
...,...,...
95959,301575470,75
95960,301575470,1
95966,308250127,14
95967,308250127,14


## Translate top 50 DX_Code Integer encoding to one hot encoding

In [41]:
id_encounter_one_hot = {}
size_of_DX_code = len(encounters_encoded_top50["DX_CODE_ENCODED"].unique())
for index, row in encounters_encoded_top50.iterrows():
    patient_id = int(row[0])
    DX_code_int = int(row[1])
    if patient_id in id_encounter_one_hot: #one hot encoding
        id_encounter_one_hot[patient_id][DX_code_int] = 1
    else: #initialize vector of zero
        id_encounter_one_hot[patient_id] = np.zeros(size_of_DX_code)

## Pharmacy Data

In [42]:
pharmacy_raw_df.isna().sum()

STUDY_ID                     0
REFILL_TIME                  0
NDC                          0
GPI_GENERIC_NAME             0
NAME_DRG_GROUP               0
NAME_DRG_CLASS               0
NAME_DRG_SUB_CLASS           0
NUMBER_OF_DAYS_SUPPLY        0
DISPENSE_AMOUNT              0
GIVE_STRENGTH            13080
dtype: int64

## Translate NAME_DRUG_GROUP to Integer encoding

In [43]:
unique_drug_group_arr = pharmacy_raw_df["NAME_DRG_GROUP"].unique() 
unique_drug_group_dict = {} #make a DX_CODE to integer dictionary
for i,v in enumerate(unique_drug_group_arr):
    unique_drug_group_dict[v] = i
    
def encoding_drug_group(v):
    return unique_drug_group_dict[v]

pharmacy_raw_df["DRUG_GROUP_ENCODED"] = pharmacy_raw_df["NAME_DRG_GROUP"].apply(lambda x: encoding_drug_group(x))
pharmacy_raw_df

,STUDY_ID,REFILL_TIME,NDC,GPI_GENERIC_NAME,NAME_DRG_GROUP,NAME_DRG_CLASS,NAME_DRG_SUB_CLASS,NUMBER_OF_DAYS_SUPPLY,DISPENSE_AMOUNT,GIVE_STRENGTH,DRUG_GROUP_ENCODED
0,19492,18MAY2017,46087221,"Estrogens, Conjugated Vaginal Cream 0.625 MG/GM",VAGINAL AND RELATED PRODUCTS,Vaginal Estrogens,Vaginal Estrogens,30,30.0,0.625,0
1,19492,31JUL2017,13107015490,Paroxetine HCl Tab 10 MG,ANTIDEPRESSANTS,Selective Serotonin Reuptake Inhibitors (SSRIs),Selective Serotonin Reuptake Inhibitors (SSRIs),30,30.0,10.000,1
2,19492,14AUG2017,43547036009,Losartan Potassium Tab 25 MG,ANTIHYPERTENSIVES,Angiotensin II Receptor Antagonists,Angiotensin II Receptor Antagonists,30,30.0,25.000,2
3,19492,28AUG2017,13107015490,Paroxetine HCl Tab 10 MG,ANTIDEPRESSANTS,Selective Serotonin Reuptake Inhibitors (SSRIs),Selective Serotonin Reuptake Inhibitors (SSRIs),30,30.0,10.000,1
4,19492,11SEP2017,43547036009,Losartan Potassium Tab 25 MG,ANTIHYPERTENSIVES,Angiotensin II Receptor Antagonists,Angiotensin II Receptor Antagonists,30,30.0,25.000,2
...,...,...,...,...,...,...,...,...,...,...,...
58325,301575470,19NOV2018,55111029336,Sumatriptan Succinate Tab 100 MG,MIGRAINE PRODUCTS,Serotonin Agonists,Selective Serotonin Agonists 5-HT(1),30,9.0,100.000,34
58326,301575470,14DEC2018,55111029336,Sumatriptan Succinate Tab 100 MG,MIGRAINE PRODUCTS,Serotonin Agonists,Selective Serotonin Agonists 5-HT(1),30,9.0,100.000,34
58327,301575470,28JAN2019,55111029336,Sumatriptan Succinate Tab 100 MG,MIGRAINE PRODUCTS,Serotonin Agonists,Selective Serotonin Agonists 5-HT(1),5,9.0,100.000,34
58328,301575470,29JAN2019,378181377,Levothyroxine Sodium Tab 125 MCG,THYROID AGENTS,Thyroid Hormones,Thyroid Hormones,90,90.0,125.000,31


In [44]:
pharmacy_encoded = pharmacy_raw_df[["STUDY_ID", "DRUG_GROUP_ENCODED"]]
pharmacy_encoded

,STUDY_ID,DRUG_GROUP_ENCODED
0,19492,0
1,19492,1
2,19492,2
3,19492,1
4,19492,2
...,...,...
58325,301575470,34
58326,301575470,34
58327,301575470,34
58328,301575470,31


## Integer Encode the top 50 most common drug group

In [45]:
pharmacy_encoded_top50 = integer_encode_top_k_variable(pharmacy_encoded, "DRUG_GROUP_ENCODED", 75)
pharmacy_encoded_top50 

/var/folders/72/zllm1_k55v9809qz_mgkcq6m0000gn/T/ipykernel_12110/3776220918.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[index, col] = dic[row_val]
/var/folders/72/zllm1_k55v9809qz_mgkcq6m0000gn/T/ipykernel_12110/3776220918.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[index, col] = k


Top  75  labels cover  0.9991428081604663 % of all the values in  DRUG_GROUP_ENCODED


,STUDY_ID,DRUG_GROUP_ENCODED
0,19492,47
1,19492,0
2,19492,2
3,19492,0
4,19492,2
...,...,...
58325,301575470,25
58326,301575470,25
58327,301575470,25
58328,301575470,4


## Translate top 50 NAME_DRUG_GROUP Integer encoding to one hot encoding

In [46]:
id_drug_one_hot = dict()
size_of_drug_group_code = len(pharmacy_encoded_top50["DRUG_GROUP_ENCODED"].unique())
for index, row in pharmacy_encoded_top50.iterrows():
    patient_id = row[0]
    drug_code_int = row[1]
    if patient_id in id_drug_one_hot: #one hot encoding
        id_drug_one_hot[patient_id][drug_code_int] = 1
    else: #initialize vector of zero
        id_drug_one_hot[patient_id] = np.zeros(size_of_drug_group_code)

In [47]:
#id_drug_one_hot R^8
#id_encounter_one_hot R^5051
#id_demographics R^84

In [48]:
x = np.concatenate((id_drug_one_hot[19492],id_encounter_one_hot[19492], id_demographics_train[19492]) )
x.size

156

## Concatenate 3 vectors: demographics, diagnosis, pharma

In [49]:
all_patient_id_train = demographic_train.STUDY_ID.unique()
id_concat_train = dict()
for id_ in all_patient_id_train:
    if (id_ in id_drug_one_hot) and (id_ in id_encounter_one_hot) and id_ in (id_demographics_train):
        id_concat_train[id_] = np.concatenate((id_demographics_train[id_], id_drug_one_hot[id_],id_encounter_one_hot[id_]))
    
        

In [50]:
all_patient_id_test = demographic_test.STUDY_ID.unique()
id_concat_test = dict()
for id_ in all_patient_id_test:
    if (id_ in id_drug_one_hot) and (id_ in id_encounter_one_hot) and id_ in (id_demographics_test):
        id_concat_test[id_] = np.concatenate((id_demographics_test[id_], id_drug_one_hot[id_],id_encounter_one_hot[id_]))


In [51]:
X_train = pd.DataFrame.from_dict(id_concat_train, orient = "index")
X_train

,0,1,2,3,4,5,6,7,8,9,...,146,147,148,149,150,151,152,153,154,155
677937,0.0,0.142857,0.296724,0.142857,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3072656,0.0,0.000000,0.217726,0.285714,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
693298,0.0,0.000000,0.055877,0.571429,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4101963,0.0,0.000000,0.123314,0.571429,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
46013841,0.0,0.142857,0.360308,0.285714,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217618,0.0,0.142857,0.400771,0.428571,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1305551,0.0,0.000000,0.219653,0.285714,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3522122,0.0,0.000000,0.321773,0.142857,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
247226404,1.0,0.000000,0.223507,0.142857,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [52]:
X_test = pd.DataFrame.from_dict(id_concat_test, orient = "index")
X_test

,0,1,2,3,4,5,6,7,8,9,...,146,147,148,149,150,151,152,153,154,155
3652678,0.0,0.142857,0.855491,0.714286,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
42136830,0.0,0.000000,0.169557,0.142857,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
143215,0.0,0.000000,0.159923,0.857143,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7195996,0.0,0.000000,0.084778,0.142857,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3293091,1.0,0.000000,0.123314,0.714286,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396791,0.0,0.000000,0.406551,0.571429,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3260152,0.0,0.000000,0.317919,0.571429,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
728094,0.0,0.142857,0.157996,0.714286,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5871662,0.0,0.000000,0.362235,0.285714,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
Y_train = demographic_X_df[demographic_X_df["STUDY_ID"].isin(id_adj_matrix)]
Y_train = Y_train[["STUDY_ID", "SARCOPENIA"]]
Y_train = Y_train.set_index('STUDY_ID')
Y_train = Y_train.sort_index()

In [75]:
Y_test = demographic_X_df[demographic_X_df["STUDY_ID"].isin(X_test.index)]
Y_test = Y_test[["STUDY_ID", "SARCOPENIA"]]
Y_test = Y_test.set_index('STUDY_ID')
Y_test = Y_test.sort_index()

### Export df to Excel files.

In [78]:
# X_train  #node features
# Y_train #class label
# X_test #node features
# Y_test #class label
# edge_weight #edge weight matrix
# adj_matrix  #adj matrix


In [92]:
path = cwd + "../Data/Proccessed/Y.csv"
Y_concat.to_csv(path)

path = cwd + "../Data/Proccessed/X.csv"
X_concat.to_csv(path)